# Spotify Playlist Creator Main Runner
Contains the code to extract liked songs, sort them based on energy/bpm, then create playlists based on that seperation. 


### Login to Spotify API

In [1]:
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

with open("ids.txt", 'r') as f:
    lines = f.readlines()
cid = lines[0][:-1]
secret = lines[1]
uri = 'http://localhost:8080'
#client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

#sp = sp.Spotify(client_credentials_manager = client_credentials_manager)

scope = "user-library-read playlist-modify-public ugc-image-upload"

sp = sp.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=cid, client_secret=secret, redirect_uri=uri))

In [7]:
print(cid)

9467290d15c44a54a01ee8ff2fc81aea


### Get Liked Songs

In [2]:
tracks = []
ids = []
artists = []
song_uris = []
energies = []
tempos = []
danceabilities = []
keys = []
loudnesses = []
valences = []
speechinesses = []
off = 0
while off < 10000:
    results = sp.current_user_saved_tracks(offset=off, limit=50)

    for idx, item in enumerate(results['items']):
        
        # Track Info
        track = item['track']
        tracks.append(track['name'])
        artists.append(track['artists'][0]['name'])
        ids.append(track['id'])
        song_uris.append(track['uri'])

        # Track Features
        feats = sp.audio_features(track['uri'])[0]
        energies.append(feats['energy'])
        tempos.append(feats['tempo'])
        danceabilities.append(feats['danceability'])
        keys.append(feats['key'])
        loudnesses.append(feats['loudness'])
        valences.append(feats['valence'])
        speechinesses.append(feats['speechiness'])
    
    if len(results['items']) == 0:
        break
    else:
        off += 50



KeyboardInterrupt: 

In [9]:
import pandas as pd
track_frame = pd.DataFrame(data = list(zip(ids, tracks, artists, energies, tempos, danceabilities, keys, loudnesses, valences, speechinesses, song_uris)), 
    columns = ['ID', 'TrackName', 'Artist', 'Energy', 'Tempo', 'Danceability', 'Key', 'Loudness', 'Valence', 'Speechiness', 'URI'])
track_frame.to_csv("NewPlaylistCSV.csv")


In [10]:
new_playlist_frame = track_frame[(track_frame['Energy'] > 0.75) | ((track_frame['Danceability'] > 0.75) & (track_frame['Energy'] > 0.65))]


In [9]:
new_playlist_frame = track_frame[(track_frame['Energy'] < 0.4) & (track_frame['Loudness'] < -6)]

In [ ]:
new_playlist_frame.to_csv("NewPlaylistCSV.csv")

In [10]:

user = sp.current_user()
user_id = user['id']
user_name = user['display_name']
name_to_create = user_name + "'s " + 'Mild Auto Generated'
user_playlists = sp.user_playlists(user_id)['items']
# Check if playlist already exists and delete if it does
there = None
for playlist in user_playlists:
    if name_to_create == playlist['name']:
        there = playlist['id']
        break
if there != None:
    sp.user_playlist_unfollow(user_id, there)

new_playlist = sp.user_playlist_create(user_id, name_to_create, public=True, collaborative=False, description="Auto Generated Mild playlist from my liked songs")
new_playlist_id = new_playlist['id']

tracks_to_add = new_playlist_frame['URI'].tolist()

for i in range(0, len(tracks_to_add), 99):
    j = i + 99
    if j > len(tracks_to_add):
        j = len(tracks_to_add)
    sp.user_playlist_add_tracks(user_id, new_playlist_id, tracks_to_add[i:j])




In [7]:
track_frame[track_frame['Artist'] == 'The Weeknd'].to_csv('weeknd.csv')
